# ARD Selects

### Before we begin: set credentials
The ARD Python SDK manages access tokens for you. 
Normally max_ard uses a configuration file to store your credentials, but for running remotely we suggest setting environment variables instead for more security.

In [87]:
# run to input your username (email address)
import os
os.environ['ARD_USERNAME'] = input()

In [88]:
# run to input your password
from getpass import getpass
os.environ['ARD_PASSWORD'] = getpass()

To run Selects from Python the SDK has a `Select` object that represents a Select and its interaction with the API.

In [89]:
from max_ard import Select

Selects require either some kind of geographic Area of Interest, or list of one or more Acquistion IDs to use. For this notebook we'll use a bounding box over Albuquerque, New Mexico. We'll also add a date range.

In [90]:
bbox = [-106.8, 35.1, -106.4, 35.4]
datetime =  "2020-07-01T00:00:00Z/2021-01-25T00:00:00Z"

The Select System also lets you filter on metadata fields. The Select API expects filter queries in this JSON object structure (here as a Python dictionary). For a full list of properties and operators you can use to build queries, see the [Select API documentation](https://ard.maxar.com/docs/api-reference/post-select/)

In [91]:
query = {
        "platform": {
          "eq": "worldview-02"
        },
        
        "aoi:cloud_free_percentage": {
          "gte": 95
        },
        "aoi:data_percentage": {
          "gte": 75
        }
    }

This query sets three filters for the select system to use to pick tiles. They are, in order:

- The platform (a satellite or constellation of satellites) must be Worldview 2
- The tile must be at least 95% free of clouds within the supplied Area of Interest (the `bbox`)
- The AOI must also be at least 75% full of valid data pixels

The call signature for a Select includes the following keyword arguments:
    
- `acq_ids`: An iterable of acquisition IDs to search for.
- `datetime`: A datetime string or range:
    - single date: `“2020-10-26T13:00:00Z”`
    - range: _use a slash_ `“2019-10-25T09:00:00Z/2020-10-26T13:00:00Z”`
    - open-ended range: _use two dots and a slash_ `“../2020-10-26T13:00:00Z”`
- `intersects`: A WGS-84 geometry to limit the search to. Can be WKT or a Shapely geometry.
- `bbox`: A 4-tuple of the WGS84 bounding box to limit the search to, in the form [XMIN, YMIN, XMAX, YMAX]
- `query`: API query JSON as a Python dictionary
- `stack_depth`: Limits the number of tiles returned per grid cell. When searching the tiles are ranked and the best scoring tiles will be used to return this number of tiles. Default is 5 tiles.

We'll create a Select using the `datetime`, `bbox`, and `query` we defined. We'll also add a stack depth of 3.

In [92]:
s = Select(datetime=datetime, bbox=bbox, query=query, stack_depth=3)

Let's take a look at `Select`s:


### Properties of the Select object

A Select has the following properties after it is created:

- `request`: A container object storing the request options.

After the Select has been submitted with `.submit()`, the following properties can be used:

- `select_id`: The Select ID assigned to the request
- `finished`: Boolean if the Select has finished running
- `state`: state of the select process: 'RUNNING', 'SUCCEEDED', or 'FAILED'


Once the Select has finished, these properties are available:

- `response`: A container object of the API response
- `usage`: A convenience property for the data usage section of the Select response
- `results`: A SelectResults object that for examining the Select response

### Methods of the Select object

- `submit()`: submits the select
- `get_link_contents(name)` Gets the contents of one of the results files, where `name` can be `stac`, `html`, `geojson`, or `geojsonl`
- `copy_file(name, dir='.')` Copies a result file locally, where `name` can be `stac`, `html`, `geojson`, or `geojsonl`. The file's name will be `<select id>.<name>`, and can be written to the optional location `dir`. If `dir` is not provided it will default to the current working directory.
- `get_signed_link(name)` Gets a signed link for a result file, where `name` can be `stac`, `html`, `geojson`, or `geojsonl`. Note: do not use an authenticated session from `get_session()` to download signed links. 

Since we have not submitted the request yet, the only property we can access is the `request` we are going to send. Trying to access other properties or methods will raise a `NotSubmitted` error.

In [93]:
from max_ard.exceptions import NotSubmitted
print(s.request)
try:
    print(s.state)
except NotSubmitted:
    print('The Select has not been submitted yet and does not have a state')

ids=None datetime='2020-07-01T00:00:00Z/2021-01-25T00:00:00Z' stack_depth=3 intersects=None bbox=[-106.8, 35.1, -106.4, 35.4] query={'platform': {'eq': 'worldview-02'}, 'aoi:cloud_free_percentage': {'gte': 95}, 'aoi:data_percentage': {'gte': 75}} image_age_category=None
The Select has not been submitted yet and does not have a state


We can submit the Select query. Once the Select is submitted to the API it will have a `state` and a `select_id`. Select requests will try to return a response within 20 seconds. If the results can not be computed in that time only a job number is returned. The `Select` object handles this internally. 

In [94]:
s.submit()
print(s.state)
print(s.select_id)

ReadTimeout: HTTPSConnectionPool(host='ard.maxar.com', port=443): Read timed out. (read timeout=None)

`Select.wait_for_success()` will poll the API to see if the Select has completed:

In [ ]:
s.wait_for_success()
print(f'Select {s.select_id} is complete')

To create a `Select` object from ID, use the `from_id` classmethod:

```
s = Select.from_id('5629729628519955012')
<ARD Select 5629729628519955012>
```

### Select Result Files

A completed Select process generates output files. These are:

- `.html`: an interative map of the results
- `.geojson`: A GeoJSON FeatureCollection of ARD grid tiles and which acquisitions fulfill the query for that cell
- `.geojsonl`: A GeoJSON Line representation of the above
- `.stac`: A Spatio-Temporal Asset Catalog (STAC) of all the ARD tiles found in the search


To access the contents of one of the results files, use `get_link_contents(name)`:

In [ ]:
geojson = s.get_link_contents('geojson')
print(geojson[:100] + '...')

To copy the file locally, use `copy_file(name)`:

In [ ]:
from os import listdir
s.copy_file('geojson')
listdir('.')

You can also get a temporary signed link for any result file. Note that these signed links should not be downloaded from an authenticated session.


In [ ]:
link = s.get_signed_link('html')
print(link)

### Data Usage

You can find the data usage a Select would incur if ordered in `.usage`

In [ ]:
s.usage

### Select Results

The `Results` object provides convenient access to Select results.

In [ ]:
results = s.results
print(results)

Results support the Python Geospatial interface and can be loaded by Shapely and other libraries

In [ ]:
from shapely.geometry import shape
shape(results)

Results have a list of dates, as well as start and end dates:

In [ ]:
results.dates

In [ ]:
results.start_date, results.end_date

To access all the tiles in the results:

In [ ]:
results.tiles

A `stack` is a collection of all of the ARD tiles covering a given cell. We can look at all of the stacks via `.stacks`:

In [ ]:
results.stacks

An `Acquisition` is a collection of all of the ARD tiles from a common acquisition. All of the acquisitions in a stack can be accessed via `.acquisitions`:

In [ ]:
results.acquisitions

### Acquisitions

`Acquisitions` is a list and can be iterated or indexed. You can also get a specific acquisition from the results using `get_acquisition()` and an acquisition ID.

In [ ]:
acquisition = results.acquisitions[0]
acquisition

In [ ]:
results.get_acquisition('10300100B3841C00')

`Acquisition` objects also have a shorter `str` representation:

In [ ]:
print(acquisition)

They also support the Python Geospatial Interface:

In [ ]:
shape(acquisition)

As well as have some summary properties:

In [ ]:
acquisition.properties

### Stacks

Stacks have similar qualities with the exception of `properties`:

In [ ]:
stack = results.stacks[0]
stack

In [ ]:
results.get_stack('Z13-031133231113')

In [ ]:
print(stack)

In [ ]:
shape(stack)

### Tiles

A `SelectTile` object represents an ARD Tile picked by the Select service.

`Stack` and `Acquisition` objects are also lists, and you can also get a specific tile from a SelectResult by acquisition ID and cell:

In [ ]:
tile = stack[0]
tile

In [ ]:
results.get_tile('10300100B3841C00', 'Z13-031133231113')

You can also get tiles from `Stack`s and `Acquisition`s:

In [ ]:
print(stack.get_tile_from_acquisition('10300100B3841C00'))
print(acquisition.get_tile_from_cell('Z13-031133231113'))    

A `SelectTile` has attributes for the acquisition ID, the `maxar_canvas_grid` `Cell` object it covers, and a `properties` with image metadata. Like the other objects it supports the Python Geospatial Protocol:

In [ ]:
tile.acq_id, tile.cell

In [ ]:
tile.properties

In [ ]:
shape(tile)